## Import packages needed for all of these API Calls

In [1]:
import requests
import math
import pprint
import json

# Creating a function to duplicate a job from one environment to another 

In [9]:
import requests
import json

# Creating a function to pull back job details for a given dbt Cloud job
def get_dbt_job_details(account_id, job_id, api_key):
    
    # Headers for authentication
    headers = {
        'Authorization': f'Token {api_key}',
        'Content-Type': 'application/json'
    }
    
    # get job endpoint setup
    get_job_url = f"https://cloud.getdbt.com/api/v2/accounts/{account_id}/jobs/{job_id}/"
    
    # HTTP GET request
    get_job_response = requests.get(get_job_url, headers=headers)
    
    # Checking the response status code
    if get_job_response.status_code == 200:
        return get_job_response.json()['data']
    else:
        return {'error': 'Failed to fetch job details', 'status_code': response.status_code, 'details': response.text}


# Creating a function to create a dbt cloud job
def create_dbt_job(account_id, api_key, job_data):
    
    # URL for creating a job
    url = f"https://cloud.getdbt.com/api/v2/accounts/{account_id}/jobs/"

    # Headers including the authorization token
    headers = {
        "Authorization": f"Token {api_key}",
        "Content-Type": "application/json"
    }

    # Make the POST request to create a new job
    response = requests.post(url, headers=headers, data=json.dumps(job_data))

    # Check if the request was successful
    if response.status_code == 201:
        return response.json()  # Return the JSON response from dbt Cloud
    else:
        # Return a JSON object with error information
        return {
            "error": "Failed to create job",
            "status_code": response.status_code,
            "message": response.text,
            "full response": response
        }

# defining a function to duplicate a job from one dbt Cloud environment to another
def duplicate_dbt_job_to_another_environment(account_id, api_key, id_of_job_to_duplicate, id_of_environment_to_duplicate_job_in, duplicated_job_name=None):

    # get details of the existing job
    existing_job_details = get_dbt_job_details(account_id, id_of_job_to_duplicate, api_key)

    # update the current job information
    existing_job_details['environment_id'] = id_of_environment_to_duplicate_job_in
    existing_job_details['id'] = None

    # update the job name if needed
    if duplicated_job_name is not None:
        existing_job_details['name'] = duplicated_job_name

    # remove any unneeded fields
    data_points_to_remove = ['created_at', 'updated_at', 'deactivated', 'run_failure_count', 'cron_humanized', 'next_run', 'next_run_humanized', 'raw_dbt_version', 'is_deferrable']
    for key in data_points_to_remove:
        existing_job_details.pop(key, None) 

    # create the duplicated job in the new environment
    new_job_details = create_dbt_job(account_id, api_key, existing_job_details)

    # return the new job details
    return new_job_details

In [10]:
# Example usage
api_key = '<< dbt cloud token goes here >>'
account_id = 12345  # Replace with your account ID
id_of_job_to_duplicate = 456456
id_of_environment_to_duplicate_job_in = 678910

duplicated_job_info = duplicate_dbt_job_to_another_environment(account_id, 
                                                               api_key, 
                                                               id_of_job_to_duplicate, 
                                                               id_of_environment_to_duplicate_job_in, 
                                                               duplicated_job_name = 'Airport Data Refresh - QA')

In [ ]:
# log duplicated job info
pprint.pp(duplicated_job_info)

# END OF SCRIPT